In [ ]:
report_columns = ['ma_short','ma_mid','total_profit',
                  'win_rate','loss_times','max_retrace',
                  'deal_count','win_count',
                  'period_score','retrace_score']
class Account:
    init_money = 0
    start_date = ''
    account_money = 0
    account_shares = 0
    last_buy_price = 0
    last_buy_date = ''
    floating_profits = []
    operation_detail= pd.DataFrame(columns=['date','profit_rate','account_money'])
    daily_log=True
    
    def __init__(self,init_money,start_date,short,mid):
        self.init_money = init_money
        self.account_money = init_money
        self.start_date = start_date
        self.short = short
        self.mid = mid
        
        self.account_shares = 0
        self.last_buy_price = 0
        self.last_buy_date = ''
        self.floating_profits = []
        
        self.i = 0
        self.operation_detail = pd.DataFrame(columns=['date','profit_rate','account_money'])
        self.operation_detail.loc[self.i] = [start_date,0,self.init_money]
        self.i +=1
        
        self.daily_log=True
        
    
    def open_opsition(self,date,price):
        buy_shares = buy_how_many(self.account_money,price)
            
        if(self.daily_log==True):
            if(debug):
                print(date+' BUY price:'+ tostr(price) + ' shares:'+tostr(buy_shares)+ ' space:'+tostr(buy_shares*price*100/self.account_money))

        self.account_shares += buy_shares
        self.account_money -= buy_shares*price
        self.last_buy_price = price
        self.last_buy_date = date
        
    def sell_all(self,date, price, sell_type='sell_point'):
        sell_shares = sell_how_many(self.account_shares)
            
        self.account_shares -= sell_shares
        self.account_money += sell_shares*price
        
        profit_rate = (price-self.last_buy_price)*100/self.last_buy_price
        if(self.daily_log==True):
            if(debug):
                print(date +' '+sell_type+' ' +tostr(sell_shares)+' shares, profit rate:'+tostr(profit_rate)+'%')
                print('floating_profits:'+str(self.floating_profits))
                print()
            
        self.operation_detail.loc[self.i] = [date,profit_rate,self.account_money]
        self.i +=1
        self.floating_profits.clear()
            
    def daily_check(self,date,price):
        if(is_buy_point(date,self.short,self.mid)):
            if(self.account_shares==0):
                self.open_opsition(date,price)
                
        if(self.account_shares>0):
            floating_profit_percent = (price-self.last_buy_price)*100/self.last_buy_price
            self.floating_profits.append(round(floating_profit_percent,2))
            
#             if(is_stop_profit(self.floating_profits)):
#                 self.sell_all(date,price,'stop_profit')
            
            if(is_sell_point(date,self.short,self.mid)):
                self.sell_all(date,price,'sell_point')
                
            if(is_stop_loss(date,self.last_buy_date,self.last_buy_price)):
                self.sell_all(date,price,'stop_loss')
    
    def max_retrace(self):
        max_retrace_loss = 0
        
        moneys = self.operation_detail['account_money']
        for i in range(1,len(moneys)):
            history_part = moneys[:i+1]
            max_money = max(history_part)
            
            if(moneys[i]<max_money):
                retrace_percent = (max_money-moneys[i])*100/max_money
                
                if(retrace_percent>max_retrace_loss):
                    max_retrace_loss = retrace_percent

        return max_retrace_loss

    
    def continue_loss(self):
        max_loss_count = 0
        
        profits = self.operation_detail['profit_rate']
        for i in range(1,len(profits)):
            cur_profit = profits[i]
            
            if(cur_profit<=0):
                loss_count = 0
                j=i;
                while(j>=0 and profits[j]<=0):
                    loss_count+=1
                    j -= 1
                if(loss_count>max_loss_count):
                    max_loss_count = loss_count
                
        return max_loss_count
        
    def get_last_year_remain_money(self, year, the_first_year):
        if(year==the_first_year):
            return self.init_money
        
        last_year = year-1
        last_year_operations = self.operation_detail[self.operation_detail['year']==last_year]
        
        if(last_year_operations.shape[0]>0):
            return last_year_operations.loc[last_year_operations.shape[0]-1:]['account_money'].values[-1]
        else:
            return self.get_last_year_remain_money(last_year, the_first_year)
    
    def profit_per_year_period(self):
        self.operation_detail['year'] = self.operation_detail['date'].apply(lambda x: int(x.split('-')[0]))

        years_list = list(set(self.operation_detail['year'].values))
        
        years_list.sort()
        the_first_year = years_list[0]
        
        period_rates = []
        for i in range(0, len(years_list)):
            year = years_list[i]

            part = self.operation_detail[self.operation_detail['year']==year]
            remain_money = part.loc[part.shape[0]-1:]['account_money'].values[-1]
            
            last_year_remain_money = self.get_last_year_remain_money(year,the_first_year)
            
            growth_rate = (remain_money-last_year_remain_money)*100/last_year_remain_money
            period_rates.append(growth_rate)

        return period_rates
        
    
    def report(self):
        total_profit_per = (self.account_money-self.init_money)*100.0/self.init_money
        
        win_count = self.operation_detail[self.operation_detail['profit_rate']>0].shape[0]
        deal_count = self.operation_detail.shape[0]
        
        win_rate = win_count*100/deal_count
        continue_loss_times = self.continue_loss()
        max_retrace = self.max_retrace()
        
        period_rates = self.profit_per_year_period()
        period_profit_score = get_score_by_year_period_profits(period_rates)
        
        retrace_score = get_score_by_max_retrace(max_retrace)
        
        report_list = [self.short,self.mid,total_profit_per,
                       win_rate,continue_loss_times,max_retrace,
                       deal_count,win_count,
                       period_profit_score,retrace_score]
        
        return report_list
    


    def buy_and_sell(start_date,end_date,report_summary, short,mid):
        print('start go through MA(' + str(short) + ',' + str(mid) + ')')
        all_dates = data['date'][mid:]

        account = Account(initial_money,start_date,short,mid)

        start_year_s, start_mon_s, start_day_s = start_date.split('-')
        start_datetime = datetime(int(start_year_s), int(start_mon_s), int(start_day_s))

        end_year_s, end_mon_s, end_day_s = end_date.split('-')
        end_datetime = datetime(int(end_year_s), int(end_mon_s), int(end_day_s))

        for date in all_dates:
            if(not(out_of_range(date,start_datetime,end_datetime))):
                price = data[data['date']==date]['close'].values[0]
                account.daily_check(date,price)

        report_data = account.report()
        report_summary.loc[report_summary.shape[0]+1] = report_data

        return report_summary
        